In [1]:
import os, struct
from array import array as pyarray
from numpy import array, zeros
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
def load_mnist(dataset="training", digits=np.arange(10), path="."):
    #Loads MNIST files into 3D numpy arrays 
    if dataset == "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset == "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    flbl = open(fname_lbl, 'rb')
    magic_nr, size = struct.unpack(">II", flbl.read(8))
    lbl = pyarray("b", flbl.read())
    flbl.close()

    fimg = open(fname_img, 'rb')
    magic_nr, size, rows, cols = struct.unpack(">IIII", fimg.read(16))
    img = pyarray("B", fimg.read())
    fimg.close()

    ind = [ k for k in range(size) if lbl[k] in digits ]
    N = len(ind)

    images = zeros((N, rows, cols) )
    labels = zeros((N ) )
    for i in range(len(ind)):
        images[i] = array(img[ ind[i]*rows*cols : (ind[i]+1)*rows*cols ]).reshape((rows, cols))
        labels[i] = lbl[ind[i]]

    return images, labels

In [3]:
train_x, train_y = load_mnist('training')
test_x, test_y = load_mnist('testing')

In [4]:
# chekcing the dimension of dataset
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [5]:
#Reshaping the dataset
x_train = np.reshape(train_x,(train_x.shape[0],-1))
x_test = np.reshape(test_x,(test_x.shape[0],-1))
ohe = OneHotEncoder(n_values=10)
train_y = ohe.fit_transform(train_y.reshape(-1,1)).toarray()
test_y = ohe.fit_transform(test_y.reshape(-1,1)).toarray()

#normalising the dataset
x_train = x_train/255
x_test = x_test/255
x_train.shape, x_test.shape, train_y.shape, test_y.shape

((60000, 784), (10000, 784), (60000, 10), (10000, 10))

In [6]:
learning_rate = 0.01

In [7]:
#Bias variables with zero
bias = np.zeros(1)

In [8]:
#softmax function for activation
def softmax(z):
    predict_y = []
    predict = []
    for i in z:
        shifti = i-np.max(i)
        exps = np.exp(shifti)/np.sum(np.exp(shifti))
        predict_y.append(np.argmax(exps))
        predict.append(exps)
    return predict,predict_y

In [9]:
#softmax derivative for gradient
def softmax_gradient(s):
    s = np.array(s)
    s_gradient = s.reshape(-1,1)
    return np.diagflat(s_gradient) - np.dot(s_gradient,s_gradient.T)        

In [10]:
#mean squared error
def mse(batch_y,predict_y):
    predict_y = np.array(predict_y)
    predict_y = ohe.fit_transform(predict_y.reshape(-1,1)).toarray()
    return batch_y - predict_y

In [11]:
#model for training
def model(batch_x, batch_y,weight):
    predict, predict_y = softmax(np.dot(batch_x,weight)+bias)
    s_derivative = softmax_gradient(predict)
    loss = mse(batch_y,predict_y)
    gradient = np.dot(batch_x.T,loss)
    del_w = learning_rate*gradient
    #update weights
    weight = weight + del_w  
    return weight

In [12]:
#random normal initialisation for weights
#seed random no. for reproducibility
np.random.seed(1)
weight = np.random.rand(x_train.shape[1],10)
weight.shape
#training the model in batch of 20
batchsize = 20
for i in range(100): #100epochs
    for j in range(len(x_train)-20):
        batch_x = x_train[j:j+20]
        batch_y = train_y[j:j+20]
        model(batch_x,batch_y,weight)
        j=j+20

In [16]:
#predict the output on test dataset
#print(weight)
predict_test, predict_testy = softmax(np.dot(x_test,weight)+bias)
predict_testy = np.array(predict_testy)
predict_testy = ohe.fit_transform(predict_testy.reshape(-1,1)).toarray()
correct_prediction = np.equal(test_y, predict_testy)
unique,count = np.unique(correct_prediction,return_counts = True)
accuracy = (np.asscalar(count[np.argwhere(unique == True)])/np.sum(count))
print(accuracy)

0.81272
